# Introduction: Constructing a data-driven gene ontology to study disease mechanisms

This notebook demonstrates how to create data-driven gene ontologies to study disease mechanisms. This notebook focuses on Fanconi Anemia (FA), a rare genetic disorder that is associated with bone marrow failure, myeloid dysplasia, and increased cancer risk. Although mutations in 20 genes are known to cause FA by dysrupting the repair of DNA damage, the existence of other FA genes and the involvement of other pathways besides DNA repair remain unclear. To discover new FA genes and pathways, this notebook executes a five-step pipeline to construct a Fanconi Anemia gene ontology (FanGO)

1. Gather input data, consisting of the 20 known FA genes as a seed set of genes for modeling and a pre-computed gene similarity network derived by integrating several types of molecular evidence including protein-protein interactions, co-expression, co-localization, and epistasis.

2. Score every gene for its involvement in FA by calculating its average functional similarity to the seed genes. The minimum score among the seed genes was used as a threshold to identify an additional set of 174 candidate genes.

3. Organize all genes in a hierarchy of 74 cellular subsystems to construct FanGO.

4. Align FanGO to the Gene Ontology.

5. Upload FanGO to an online database, the Network Data Exchange ([NDEx](http://ndexbio.org)), and visualize FanGO in the [HiView](http://hiview.ucsd.edu) web application.

Code is also provided to analyze 651 other diseases using a similar pipeline. Known gene-disease associations are taken from the [Monarch Initiative database](https://monarchinitiative.org/)

Before reading this notebook, it is recommended that you look at the [DDOT tutorial](https://github.com/michaelkyu/ddot/blob/master/examples/Tutorial.ipynb)

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/software_pipeline_23jan2018.png" width="700" align="left">

In [1]:
import pandas as pd
import networkx as nx
import numpy as np

import ddot
from ddot import Ontology

# Set username and password on the Network Data Exchange (NDEx).
* It is strongly recommended that you create a free account on NDEx in order to keep track of your own ontologies.
* Note that there are two NDEx servers: the main server at http://public.ndexbio.org/ and a test server for prototyping your code at http://test.ndexbio.org (also aliased as http://dev2.ndexbio.org). Each server requires a separate user account. Because the main server contains networks from publications, we recommend that you use an account on the test server while you become familiar with DDOT

In [2]:
# Note: change the server to http://public.ndexbio.org, if this is where you created your NDEx account
ndex_server = 'http://test.ndexbio.org' 

# Set the NDEx server and the user account (replace with your own account)
ndex_user, ndex_pass = '<enter your username>', '<enter your account password>'

# Read gene-gene integrated similarity network

In [ ]:
# Install the simplejson package (it is recommend you run this in a separate bash terminal, not in this Jupyter notebook. If you want to use a conda virtual environment, then you first need to activate the environment)
! pip install simplejson

In [3]:
## Download human gene-gene similarity network from NDEx
## -- WARNING: This network is very large (19,009-by-19,009 matrix).
## -- *** Requires ~6 GB of RAM to download and process ***
## -- *** ~4 GB of data is downloaded from NDEx, which takes ~10 min for a fast internet connection ***
## -- This requires the simplejson package (see previous cell)
sim, sim_names = ddot.ndex_to_sim_matrix(
    ndex_url=ddot.HUMAN_GENE_SIMILARITIES_URL,
    input_fmt='cx_matrix',
    output_fmt='matrix')
sim = pd.DataFrame(sim, columns=sim_names, index=sim_names)

## Rank transform the similarities
sim_rank = sim.rank(0) / (sim.shape[0] - 1)
sim_rank = pd.DataFrame((sim_rank.values + sim_rank.values.T) / 2.0, columns=sim_names, index=sim_names)

sim_rank.head()

NDEx download and CX parse time (sec): 356.2376916408539
Dim: (19009, 19009)
Bytes: 2890736648
Iterate through CX and construct array time (sec): 14.60573434829712


,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
A1BG,0.000053,0.654751,0.466593,0.322811,0.027936,0.451599,0.332334,0.221617,0.565551,0.563473,...,0.470512,0.093434,0.088042,0.922059,0.198680,0.991425,0.536011,0.479693,0.021175,0.290588
A1CF,0.654751,0.000053,0.871843,0.304977,0.281618,0.465988,0.694550,0.428951,0.291614,0.970460,...,0.334991,0.475563,0.453309,0.963410,0.237216,0.781723,0.952073,0.453467,0.756576,0.301505
A2M,0.466593,0.871843,0.000053,0.999053,0.062145,0.820181,0.777252,0.430240,0.480298,0.816577,...,0.327862,0.455414,0.204230,0.495449,0.205282,0.255577,0.262837,0.731297,0.850221,0.379972
A2ML1,0.322811,0.304977,0.999053,0.000053,0.015770,0.474248,0.333386,0.476089,0.438315,0.346933,...,0.189578,0.276015,0.260390,0.968592,0.047822,0.890704,0.282092,0.447627,0.663615,0.100694
A3GALT2,0.027936,0.281618,0.062145,0.015770,0.000053,0.049229,0.009654,0.083807,0.155461,0.193261,...,0.318918,0.377512,0.259443,0.514783,0.197062,0.574311,0.825521,0.335596,0.608165,0.063526


# Specify a set of seed genes with known associations to the disease being studied

In [4]:
# Let seed genes be the 20 known genes that cause Fanconi Anemia (from the Fanconi Anemia Mutation Database, http://www2.rockefeller.edu/fanconi/)
seed = ['FANCA', 'FANCB', 'FANCC', 'BRCA2', 'FANCD2',
        'FANCE', 'FANCF', 'FANCG', 'FANCI', 'BRIP1',
        'FANCL', 'FANCM', 'PALB2', 'RAD51C', 'SLX4',
        'ERCC4', 'RAD51', 'BRCA1', 'UBE2T', 'XRCC2']

In [ ]:
# # Let seed genes be the known genese for one of 651 diseases (uncomment to use)

# # Retrieve a table of gene-disease associations from the Monarch Initiative (reformatted and stored on NDEx)
# monarch, _ = ddot.ndex_to_sim_matrix(
#     ddot.config.MONARCH_DISEASE_GENE_SLIM_URL,
#     input_fmt='cx',
#     output_fmt='sparse')
# print(monarch.head())

# # Example: get the known genes for Caffey Disease
# seed = monarch.loc[monarch['disease']=='caffey_disease', 'gene'].tolist()
# seed = [s for s in seed if s in sim_names]
# print('Seed:', seed)

# Identify candidate set of genes that are highly similar to the seed set of genes

In [5]:
expand, expand_idx, sim_2_seed, fig = ddot.expand_seed(
    seed,
    sim_rank.values,
    sim_names,
    seed_perc=0,
    agg='mean',
    figure=True)

# Organize seed and candidate genes into a data-driven gene ontology

In [6]:
# Run CliXO, with parameters alpha=0.05 and beta=0.5
ont = Ontology.run_clixo(sim.loc[expand, :].loc[:, expand], alpha=0.05, beta=0.5, square=True)
ont

194 genes, 74 terms, 349 gene-term relations, 86 term-term relations
node_attributes: ['Parent weight']
edge_attributes: ['CLIXO_score']

# Align the data-driven ontology with the Gene Ontology (GO)

In [7]:
# Read Gene Ontology from NDEx. This version has been pre-processed to contain a non-redundant set of GO terms and connections that are relevant to human genes (see Get_Gene_Ontology.ipynb) 
go_human = Ontology.from_ndex(ddot.config.GO_HUMAN_URL)
print(go_human)


19015 genes, 19343 terms, 215488 gene-term relations, 36362 term-term relations
node_attributes: ['Term_Description', 'name', 'Branch', 'Vis:Fill Color', 'Vis:Border Paint', 'Vis:Shape']
edge_attributes: ['Vis:Visible']


In [8]:
# Align ontologies
alignment = ont.align(go_human, 
                      iterations=100,
                      update_self=['Term_Description'],
                      align_label='Term_Description',
                      verbose=True)
alignment.head()

Common genes: 193
collapse command: /cellar/users/mikeyu/anaconda2/envs/ddot_py36/lib/python3.6/site-packages/ddot/alignOntology/collapseRedundantNodes /tmp/tmpur4okjlk
collapse command: /cellar/users/mikeyu/anaconda2/envs/ddot_py36/lib/python3.6/site-packages/ddot/alignOntology/collapseRedundantNodes /tmp/tmpibt10j5g
ont1_collapsed: 193 genes, 74 terms, 348 gene-term relations, 86 term-term relations
node_attributes: ['Parent weight']
edge_attributes: ['CLIXO_score']
ont2_collapsed: 193 genes, 1854 terms, 3594 gene-term relations, 4490 term-term relations
node_attributes: ['Term_Description', 'name', 'Branch', 'Vis:Fill Color', 'Vis:Border Paint', 'Vis:Shape']
edge_attributes: ['Vis:Visible']
Alignment command: /cellar/users/mikeyu/anaconda2/envs/ddot_py36/lib/python3.6/site-packages/ddot/alignOntology/calculateFDRs /tmp/tmpk0eckbod /tmp/tmprb3m4od4 0.05 criss_cross /tmp/tmpczovc09t 100 40 gene


,Term,Similarity,FDR
Term,,,
S:196,GO:0035098,0.902149,0.0
S:219,GO:1905773,0.900885,0.0
S:206,GO:1901673,0.891945,0.0
S:205,GO:0004748,0.891813,0.0
S:240,GO:0071821,0.891386,0.0


In [9]:
# Note how node attributes have been updated to reflect the ontology alignment
ont.node_attr

,Parent weight,Aligned_Term,Aligned_Similarity,Aligned_FDR,Aligned_Term_Description,Label
S:194,7.90340,GO:0005873,0.441441,0.0,plus-end kinesin complex,S:194\nplus-end kinesin complex
S:195,8.22255,GO:0098535,0.889600,0.0,de novo centriole assembly involved in multi-c...,S:195\nde novo centriole assembly involved in ...
S:196,8.18291,GO:0035098,0.902149,0.0,ESC/E(Z) complex,S:196\nESC/E(Z) complex
S:197,8.13613,GO:0001882,0.450824,0.0,nucleoside binding,S:197\nnucleoside binding
S:198,8.13260,GO:0000942,0.300136,0.0,condensed nuclear chromosome outer kinetochore,S:198\ncondensed nuclear chromosome outer kine...
S:199,8.09260,GO:0000732,0.101758,0.0,strand displacement,S:199\nstrand displacement
S:200,7.99652,GO:0051087,0.505705,0.0,chaperone binding,S:200\nchaperone binding
S:201,7.99062,GO:0097472,0.447222,0.0,cyclin-dependent protein kinase activity,S:201\ncyclin-dependent protein kinase activity
S:202,7.98245,GO:0000320,0.462717,0.0,re-entry into mitotic cell cycle,S:202\nre-entry into mitotic cell cycle
S:203,7.96387,GO:1990414,0.447235,0.0,replication-born double-strand break repair vi...,S:203\nreplication-born double-strand break re...


# Upload ontology with NDEx to visualize in the HiView application (http://hiview.ucsd.edu)
* A two-dimensional layout of nodes is automatically calculated to optimize visualization of hierarchical structure
* Molecular networks, such as protein-protein interactions and RNA coexpression, can be visualized in HiView to understand how an ontology's structure is consistent with data
* Node attributes (color and size) can be set to visualize metadata.

In [10]:
# Set the node color of seed genes to be green
fill_attr = pd.DataFrame({'Vis:Fill Color' : '#6ACC65'}, index=seed)
ont.update_node_attr(fill_attr)

In [11]:
# Set the node color of inferred terms according to the alignment with GO (for visualization in HiView)
fill_attr = ont.node_attr['Aligned_Similarity'].dropna().map(ddot.color_gradient)
fill_attr = fill_attr.to_frame().rename(columns={'Aligned_Similarity' : 'Vis:Fill Color'})
ont.update_node_attr(fill_attr)

In [12]:
# Download a table containing multiple types of gene-gene interactions, which were preformatted and uploaded to NDEx for the Fanconi Anemia example.
from ndex.networkn import NdexGraph
G_ndex = NdexGraph(server=ddot.parse_ndex_server(ddot.FANGO_DATA_URL), uuid=ddot.parse_ndex_uuid(ddot.FANGO_DATA_URL))
G = ddot.NdexGraph_to_nx(G_ndex)
gene_network_data = ddot.nx_edges_to_pandas(G)
gene_network_data.index.names = ['Gene1', 'Gene2']
gene_network_data.reset_index(inplace=True)
gene_network_data['RandomForest integrated similarity'] = [sim.loc[g1, g2] for g1, g2 in zip(gene_network_data['Gene1'], gene_network_data['Gene2'])]
gene_network_data.head()

,Gene1,Gene2,PPI sum,Giant_5,GTEx Artery-Aorta,GTEx Breast-MammaryTissue,GTEx sum,PPI CCSB,GTEx Skin-SunExposed_Lowerleg,GTEx Adipose-Subcutaneous,...,Giant_4,PPI InbioMap,PPI BioGRID,PPI huMap,GTEx Skin-NotSunExposed_Suprapubic,GTEx Adipose-Visceral_Omentum,GTEx WholeBlood,GTEx Thyroid,GTEx Esophagus-Muscularis,RandomForest integrated similarity
0,BUB1B,CASP8AP2,0,0.0,False,False,0,False,False,False,...,0.0,False,False,False,False,False,False,False,False,5.472130
1,BUB1B,POLG2,0,14.0,False,False,0,False,False,False,...,70.0,False,False,False,False,False,False,False,False,6.042856
2,BUB1B,PLK4,0,106.0,False,False,3,False,False,False,...,134.0,False,False,False,False,False,False,False,False,7.227561
3,BUB1B,SRSF10,0,46.0,False,False,0,False,False,False,...,105.0,False,False,False,False,False,False,False,False,6.376164
4,BUB1B,NUDT21,0,1.0,False,False,0,False,False,False,...,4.0,False,False,False,False,False,False,False,False,6.112902


In [13]:
# Upload ontology to NDEx
ndex_url, G = ont.to_ndex(
            name='Fanconi Anemia Gene Ontology',
            description='Generated with the Data-driven Ontology Toolkit (https://github.com/michaelkyu/ddot)',
            ndex_server=ndex_server,
            ndex_user=ndex_user,
            ndex_pass=ndex_pass,
            visibility='PUBLIC',
            layout='bubble-collect',
            network=gene_network_data,
            main_feature='RandomForest integrated similarity')

print('To visualize in HiView, go to http://hiview.ucsd.edu in your web browser, and then')
print('\t--Enter this into the "NDEx Sever URL" field: %s' % ddot.parse_ndex_server(ndex_url))
print('\t--Enter this into the "UUID of the main hierarchy" field: %s' % ddot.parse_ndex_uuid(ndex_url))
print('Alternatively, go to %s' % ddot.to_hiview_url(ndex_url))


To visualize in HiView, go to http://hiview.ucsd.edu in your web browser, and then
	--Enter this into the "NDEx Sever URL" field: http://dev2.ndexbio.org/
	--Enter this into the "UUID of the main hierarchy" field: 4e8f85b7-fa70-11e8-ad43-0660b7976219
Alternatively, go to http://hiview.ucsd.edu/4e8f85b7-fa70-11e8-ad43-0660b7976219?type=test&server=http://dev2.ndexbio.org
